# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [17]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np

# Import API key
from api_keys import geoapify_key

In [18]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [19]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lat", "Lng", frame_width = 200, frame_height = 80)

# Display the map
map_plot 


:Points   [Lat,Lng]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values

cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= 20) & (city_data_df["Max Temp"] <=300) & 
                            (city_data_df["Wind Speed"] <=5) & (city_data_df["Humidity"] <=100)]
cities_df.head()

# Display sample data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
10,10,colorado,-22.8375,-51.9731,20.36,86,96,4.69,BR,1666107965
19,19,puerto ayora,-0.7393,-90.3518,22.98,82,41,1.79,EC,1666108242
20,20,avarua,-21.2078,-159.7750,22.03,73,20,2.57,CK,1666108220
22,22,guiratinga,-16.3494,-53.7583,38.35,14,22,3.10,BR,1666108243


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = narrowed_city_df
hotel_df["Hotel Name"] = np.nan
hotel_df = hotel_df.drop(columns='index')
hotel_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = cities_df
hotel_df.reset_index(drop=True, inplace=True)
hotel_df

# Display sample data


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,10,colorado,-22.8375,-51.9731,20.36,86,96,4.69,BR,1666107965
2,19,puerto ayora,-0.7393,-90.3518,22.98,82,41,1.79,EC,1666108242
3,20,avarua,-21.2078,-159.7750,22.03,73,20,2.57,CK,1666108220
4,22,guiratinga,-16.3494,-53.7583,38.35,14,22,3.10,BR,1666108243
...,...,...,...,...,...,...,...,...,...,...
181,567,sangamner,19.5667,74.2167,23.82,85,100,2.28,IN,1666108559
182,568,road town,18.4167,-64.6167,31.02,70,40,4.12,VG,1666108560
183,569,ranong,9.9658,98.6348,24.34,93,100,1.44,TH,1666108330
184,574,canchungo,12.0672,-16.0333,35.07,61,67,2.91,GW,1666108565


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [51]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "radius": 5000,
    "rankby": "prominence",
    "types": "lodging",
    "keyword": "hotel",
    "key": geoapify_key, 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Latitude']
    lng = row['Longitude']
    city = row['City']
    params["location"] = f'{lat},{lng}
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

SyntaxError: unterminated string literal (detected at line 20) (2641121758.py, line 20)

In [44]:
# Display sample data
hotel_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228,No hotel found
1,10,colorado,-22.8375,-51.9731,20.36,86,96,4.69,BR,1666107965,No hotel found
2,19,puerto ayora,-0.7393,-90.3518,22.98,82,41,1.79,EC,1666108242,No hotel found
3,20,avarua,-21.2078,-159.7750,22.03,73,20,2.57,CK,1666108220,No hotel found
4,22,guiratinga,-16.3494,-53.7583,38.35,14,22,3.10,BR,1666108243,No hotel found
...,...,...,...,...,...,...,...,...,...,...,...
181,567,sangamner,19.5667,74.2167,23.82,85,100,2.28,IN,1666108559,No hotel found
182,568,road town,18.4167,-64.6167,31.02,70,40,4.12,VG,1666108560,No hotel found
183,569,ranong,9.9658,98.6348,24.34,93,100,1.44,TH,1666108330,No hotel found
184,574,canchungo,12.0672,-16.0333,35.07,61,67,2.91,GW,1666108565,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
%%capture --no-display

# Configure the map plot
hotel_df_map = cities_df.hvplot.points("Lat", "Lng", frame_width = 200, frame_height = 80)

# Display the map
hotel_df_map

:Points   [Lat,Lng]